In [3]:
import os
print(os.getcwd())
import matplotlib.pyplot as plt
import cv2
from realsense_sensor import RealsenseSensor
import imutils
import numpy as np
import cv_utils

/home/dominik/Dokumente/python


In [4]:
cam = RealsenseSensor("realsense_config.json")

2020-01-15 23:20:28.434 | INFO     | realsense_sensor:__init__:53 - Cam <pyrealsense2.device: Intel RealSense D435 (S/N: 832112070584)> connected.


In [5]:
cam.start()

2020-01-15 23:20:29.444 | INFO     | realsense_sensor:start:97 - Realsense stream started.


In [10]:
img, d = cam.frames()
grasp_point = cv_utils.getGrasp(img, d)
robot.executeGrasp(grasp_point)

AttributeError: module 'cv_utils' has no attribute 'getGrasp'

In [7]:
intrinsics = cam.getIntrinsics()

In [9]:
intrinsics["px"]

320.8276672363281